In [6]:
import random
import numpy as np
from collections import deque

def ucitaj_graf(mtx_fajl):
    """
    Učitavanje grafa iz MTX fajla. Pretpostavlja da je graf predstavljen kao lista susedstva.
    """
    graf = {}
    with open(mtx_fajl, 'r') as f:
        linije = f.readlines()
        for linija in linije[3:]:
            if linija.strip():
                u, v = map(int, linija.split())
                if u not in graf:
                    graf[u] = []
                if v not in graf:
                    graf[v] = []
                graf[u].append(v)
                graf[v].append(u)
    return graf

def evaluacija_sekvence(graf, sekvenca):
    """
    Evaluacija sekvence: računa broj koraka potreban da se zapale svi čvorovi.
    Ako nije moguće zapaliti sve čvorove, vraća beskonačno.
    """
    zapaljeni = set()
    trenutno_zapaljeni = set()
    koraci = 0

    for cvor in sekvenca:
        koraci += 1
        # Dodaj trenutni čvor i njegove susede u zapaljene
        novo_zapaljeni = {cvor} | set(graf[cvor]) | trenutno_zapaljeni
        zapaljeni.update(novo_zapaljeni)
        trenutno_zapaljeni = novo_zapaljeni

        # Ako su svi čvorovi zapaljeni, vraća broj koraka
        if len(zapaljeni) == len(graf):
            return koraci

    # Ako nisu svi čvorovi zapaljeni
    return float('inf')
def generisi_pocetnu_sekvencu_pohlepno(graf):
    """
    Generiše početnu sekvencu koja pokriva sve čvorove koristeći pohlepni algoritam.
    """
    neposeceni = set(graf.keys())
    sekvenca = []

    # Počni od čvora sa najvećim stepenom
    trenutni_cvor = max(neposeceni, key=lambda x: len(graf[x]))
    sekvenca.append(trenutni_cvor)
    neposeceni.remove(trenutni_cvor)

    # Iterativno dodaj sledeće čvorove dok ne pokriješ sve
    while neposeceni:
        kandidati = list(neposeceni)
        najbolji_cvor = max(kandidati, key=lambda x: len(set(graf[x]) & neposeceni))
        sekvenca.append(najbolji_cvor)
        neposeceni.remove(najbolji_cvor)

    return sekvenca


def lokalna_pretraga(graf, sekvenca):
    """
    Lokalna pretraga: menja jedan ili dva čvora u sekvenci radi poboljšanja.
    """
    najbolje_rjesenje = sekvenca[:]
    najbolja_evaluacija = evaluacija_sekvence(graf, sekvenca)

    for _ in range(10):  # Isprobaj do 10 promena
        novo_rjesenje = sekvenca[:]
        # Zameni dva nasumična čvora u sekvenci
        i, j = random.sample(range(len(sekvenca)), 2)
        novo_rjesenje[i], novo_rjesenje[j] = novo_rjesenje[j], novo_rjesenje[i]

        nova_evaluacija = evaluacija_sekvence(graf, novo_rjesenje)
        if nova_evaluacija < najbolja_evaluacija:
            najbolje_rjesenje = novo_rjesenje[:]
            najbolja_evaluacija = nova_evaluacija

    return najbolje_rjesenje

def narusavanje_rešenja(sekvenca):
    """
    Narušavanje rešenja: nasumično permutuje podniz sekvence.
    """
    novo_rjesenje = sekvenca[:]
    i, j = sorted(random.sample(range(len(sekvenca)), 2))
    random.shuffle(novo_rjesenje[i:j+1])
    return novo_rjesenje

def iterated_local_search(graf, broj_iteracija):
    """
    Iterated Local Search (ILS) za Graph Burning Problem.
    """
    trenutno_rjesenje = generisi_pocetnu_sekvencu(graf)
    najbolje_rjesenje = trenutno_rjesenje[:]
    najbolja_evaluacija = evaluacija_sekvence(graf, trenutno_rjesenje)

    for _ in range(broj_iteracija):
        # Lokalna pretraga
        novo_rjesenje = lokalna_pretraga(graf, trenutno_rjesenje)
        nova_evaluacija = evaluacija_sekvence(graf, novo_rjesenje)

        if nova_evaluacija < najbolja_evaluacija:
            najbolje_rjesenje = novo_rjesenje[:]
            najbolja_evaluacija = nova_evaluacija

        # Narušavanje trenutnog rešenja
        trenutno_rjesenje = narusavanje_rešenja(novo_rjesenje)

    return najbolje_rjesenje, najbolja_evaluacija

# Glavni deo programa
if __name__ == "__main__":
    # Putanja do MTX fajla
    mtx_fajl = "grafovi/manji/karate.mtx"

    # Učitaj graf
    graf = ucitaj_graf(mtx_fajl)

    # Iterated Local Search
    broj_iteracija = 100
    najbolje_rjesenje, najbolja_evaluacija = iterated_local_search(graf, broj_iteracija)

    print("Najbolja sekvenca:", najbolje_rjesenje)
    print("Evaluacija najbolje sekvence:", najbolja_evaluacija)


Najbolja sekvenca: [34, 6, 32, 30, 1, 28, 11, 15, 9, 3, 25, 2, 14, 5, 18, 16, 10, 13, 4, 23, 8, 17, 24, 29, 27, 31, 20, 21, 7, 19, 26, 12, 33, 22]
Evaluacija najbolje sekvence: 5
